In [1]:
cd ../../data/

/Users/binyu/OneDrive - mail.bnu.edu.cn/work/data


In [2]:
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
import numpy as np

1. 读取fits文件信息
2. 制定超新星遗迹范围，并截取
3. 保存超新星遗迹数据

In [3]:
hdulist = fits.open('catwithdis.fit')

In [4]:
cols = hdulist[1].columns
cols

ColDefs(
    name = 'RA'; format = 'D'
    name = 'DEC'; format = 'D'
    name = 'J'; format = 'E'
    name = 'H'; format = 'E'
    name = 'K'; format = 'E'
    name = 'EJ'; format = 'E'
    name = 'EH'; format = 'E'
    name = 'EK'; format = 'E'
    name = 'GP'; format = 'E'
    name = 'BP'; format = 'E'
    name = 'RP'; format = 'E'
    name = 'EGP'; format = 'E'
    name = 'EBP'; format = 'E'
    name = 'ERP'; format = 'E'
    name = 'W1'; format = 'E'
    name = 'EW1'; format = 'E'
    name = 'SSID'; format = '19A'
    name = 'PARA'; format = 'E'
    name = 'EPARA'; format = 'E'
    name = 'REGK'; format = 'D'
    name = 'REBR'; format = 'D'
    name = 'REHK'; format = 'D'
    name = 'DIS'; format = 'E'
)

In [4]:
%%time 

data = hdulist[1].data

CPU times: user 12.7 ms, sys: 1.82 ms, total: 14.5 ms
Wall time: 19.2 ms


In [17]:
header = fits.getheader('catwithdis.fit',1)
header

XTENSION= 'BINTABLE'           /Binary table written by MWRFITS v1.6a           
BITPIX  =                    8 /Required value                                  
NAXIS   =                    2 /Required value                                  
NAXIS1  =                  127 /Number of bytes per row                         
NAXIS2  =             28271752 /Number of rows                                  
PCOUNT  =                    0 /Normally 0 (no varying arrays)                  
GCOUNT  =                    1 /Required value                                  
TFIELDS =                   23 /Number of columns in table                      
COMMENT                                                                         
COMMENT  *** End of mandatory fields ***                                        
COMMENT                                                                         
COMMENT                                                                         
COMMENT  *** Column names **

In [5]:
%%time

c = SkyCoord(data['ra'] * u.deg, data['dec'] * u.deg, frame='icrs')

KeyboardInterrupt: 

In [14]:
%%time

l = c.galactic.l.degree
b = c.galactic.b.degree

Wall time: 23.2 s


In [ ]:
col_l = fits.Column(name='l', format='D', array=l)
col_b = fits.Column(name='b', format='D', array=b)
new_cols = fits.ColDefs([col_l, col_b])
orig_cols = data.columns
hdu = fits.BinTableHDU.from_columns(orig_cols + new_cols)
hdu.writeto('newtable.fits')

In [6]:
hdulist.close()

In [8]:
hdulist2 = fits.open('newtable.fits')

In [10]:
cols = hdulist2[1].columns
cols

ColDefs(
    name = 'RA'; format = 'D'
    name = 'DEC'; format = 'D'
    name = 'J'; format = 'E'
    name = 'H'; format = 'E'
    name = 'K'; format = 'E'
    name = 'EJ'; format = 'E'
    name = 'EH'; format = 'E'
    name = 'EK'; format = 'E'
    name = 'GP'; format = 'E'
    name = 'BP'; format = 'E'
    name = 'RP'; format = 'E'
    name = 'EGP'; format = 'E'
    name = 'EBP'; format = 'E'
    name = 'ERP'; format = 'E'
    name = 'W1'; format = 'E'
    name = 'EW1'; format = 'E'
    name = 'SSID'; format = '19A'
    name = 'PARA'; format = 'E'
    name = 'EPARA'; format = 'E'
    name = 'REGK'; format = 'D'
    name = 'REBR'; format = 'D'
    name = 'REHK'; format = 'D'
    name = 'DIS'; format = 'E'
    name = 'l'; format = 'D'
    name = 'b'; format = 'D'
)

In [11]:
data = hdulist2[1].data

for ic443 192-186,0-6

In [17]:
index = (data.l >= 186) & (data.l <= 192) & (data.b >= 0) & (data.b <=6)

In [19]:
field = data[index]

In [ ]:
hdu = fits.BinTableHDU.from_columns(field)